# Trying to figure out exact eigenvalues for $A^{(2)}_+$

In [1]:
from sympy import *
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import linalg as la
from time import time

In [2]:
from IPython.display import display, Markdown, Latex

In [3]:
plt.rcParams['figure.figsize'] = 15, 5

In [4]:
G, nu, lam = symbols('G, nu, lam')

In [5]:
vp = G/(G+2)
ep = 2/(G+2)**2

In [6]:
sp = 0
s = sp
e = ep

In [30]:
v = s + vp
vx = (v*(v-1)+G*e).simplify()
ex = (v*(-.5*(v-1)**2+e)/nu).simplify()
ux = vx
uxx = (vx*(2*v-1)+G*ex).simplify()

g = (G*e-(nu+1)*ux)/nu
h = -ex/v
f = 2*v - 1

# Create A and $A^{(2)}_+$


In [33]:
def lift_A(a):
    to_return = [[a[0,0] + a[1,1] + a[2,2], a[2,3], a[2,4], -a[1,3], -a[1,4], 0, a[0,3], a[0,4], 0, 0],\
                 [a[3,2], a[0,0] + a[1,1] + a[3,3], a[3,4], a[1,2], 0, -a[1,4], -a[0,2], 0, a[0,4], 0],\
                 [a[4,2], a[4,3], a[0,0] + a[1,1] + a[4,4], 0, a[1,2], a[1,3], 0, -a[0,2], -a[0,3], 0],\
                 [-a[3,1], a[2,1], 0, a[0,0] + a[2,2] + a[3,3], a[3,4], -a[2,4], a[0,1], 0, 0, a[0,4]],\
                 [-a[4,1], 0, a[2,1], a[4,3], a[0,0] + a[2,2] + a[4,4], a[2,3], 0, a[0,1], 0, -a[0,3]],\
                 [0, -a[4,1], a[3,1], -a[4,2], a[3,2], a[0,0] + a[3,3] + a[4,4], 0, 0, a[0,1], a[0,2]],\
                 [a[3,0], -a[2,0], 0, a[1,0], 0, 0, a[1,1] + a[2,2] + a[3,3], a[3,4], -a[2,4], a[1,4]],\
                 [a[4,0], 0, -a[2,0], 0, a[1,0], 0, a[4,3], a[1,1] + a[2,2] + a[4,4], a[2,3], -a[1,3]],\
                 [0, a[4,0], -a[3,0], 0, 0, a[1,0], -a[4,2], a[3,2], a[1,1] + a[3,3] + a[4,4], a[1,2]],\
                 [0, 0, 0, a[4,0], -a[3,0], a[2,0], a[4,1], -a[3,1], a[2,1], a[2,2] + a[3,3] + a[4,4]]]
    return to_return

In [34]:
A = np.array([[0,1,0,0,0],\
              [lam*v/nu,v/nu,v*ux/nu-uxx,lam*g,g-h],\
              [0,0,0,lam,1],\
              [0,0,0,0,1],\
              [0,G,lam*v+G*ux,lam*v,f-lam]])

In [35]:
A2 = lift_A(A)

In [42]:
M = subMatrix(A2,[(lam,10),(G,2/3),(nu,.2)])
for i in range(10):
    print(M[i])

[ 1.25  10.     1.    -9.375 -0.938  0.     0.     0.     0.     0.   ]
[ 0.     1.25   1.     0.     0.    -0.938  0.     0.     0.     0.   ]
[ 2.5    2.5   -9.25   0.     0.     9.375  0.     0.     0.     0.   ]
[ 0.  0.  0.  0.  1. -1.  1.  0.  0.  0.]
[ -0.667   0.      0.      2.5   -10.5    10.      0.      1.      0.
   0.   ]
[  0.     -0.667   0.     -2.5     0.    -10.5     0.      0.      1.
   0.   ]
[ 0.     0.     0.    12.5    0.     0.     1.25   1.    -1.     0.938]
[ 0.     0.     0.     0.    12.5    0.     2.5   -9.25  10.    -9.375]
[ 0.    0.    0.    0.    0.   12.5  -2.5   0.   -9.25  0.  ]
[  0.      0.      0.      0.      0.      0.      0.667   0.      0.
 -10.5  ]


## Test possible eigenvalues for $A^{(2)}_+$

In [57]:
test = vp-1-lam
val = Matrix(A2-test*np.eye(10)).det().simplify()

In [59]:
val.subs([(G,2/3),(nu,.2),(lam,10)])

-37.7871093750000

# Look for Patterns

In [41]:
def subMatrix(matrix, to_sub):
    newM = np.zeros_like(matrix, dtype=float)
    for i in range(10):
        for j in range(10):
            try:
                newM[i][j] = round(float(matrix[i][j].subs(to_sub)),3)
                #print(type(newM[i][j]))
            except:
                newM[i][j] = round(matrix[i][j],3)
    return newM

In [49]:
Gvals = [.2,.4,.6,2/3]
nvals = [1,.2,.3,.4,.5,.6]
lvals = [0,10,20,30,40,50]

order1 = [(G,'gamma',Gvals), (nu,'nu',nvals), (lam,'lambda',lvals)]
order2 = [(nu,'nu',nvals), (lam,'lambda',lvals), (G,'gamma',Gvals)]
order3 = [(lam,'lambda',lvals), (G,'gamma',Gvals), (nu,'nu',nvals)]

In [47]:
def get_names(order, vals):
    for i in range(3):
        if order[i][0] == G:
            Gval = vals[i]
        elif order[i][0] == nu:
            nval = vals[i]
        else:
            lval = vals[i]
    return Gval, nval, lval

In [51]:
order = order1
for val1 in order[0][-1]:
    print(order[0][1],val1,'\n\n')
    for val2 in order[1][-1]:
        print('\t',order[1][1],val2,'\n\n')
        for val3 in order[-1][-1]:
            print('\t\t',order[-1][1],val3,'\n\n')
            
            Gval, nval, lval = get_names(order,[val1,val2,val3])
            #print(Gval,nval,lval)

            to_sub = [(order[0][0],val1),(order[1][0],val2),(order[2][0],val3)]
            M = subMatrix(A2, to_sub)
            for m in M:
                print('\t\t\t',m)
            
            evals = np.linalg.eig(M)[0]
            print('')
            print('\t\t\t',evals,'\n\n')
            
            eval0 = np.min(evals)
            print('\t\t\t','smallest eigenvalue',eval0,'\n\n')
            
            #print( '\t\t',eval0+lval+1)#,'\n\n')
            #print( '\t\t',4*2/(Gval+2)**2,'\n\n')
            print( '\t\t\tvp =',Gval/((Gval+2)),'\n\n')
            print( '\t\t\there',-lval-1-eval0,'\n\n')
            print('\t\t\t','lambda * vp', lval*Gval/(Gval+2),'\n\n')
            print('\n')

gamma 0.2 


	 nu 1 


		 lambda 0 


			 [ 0.091  0.     1.     0.    -0.083  0.     0.     0.     0.     0.   ]
			 [ 0.     0.091  1.     0.     0.    -0.083  0.     0.     0.     0.   ]
			 [ 0.     0.    -0.727  0.     0.     0.     0.     0.     0.     0.   ]
			 [ 0.  0.  0.  0.  1. -1.  1.  0.  0.  0.]
			 [-0.2    0.     0.     0.    -0.818  0.     0.     1.     0.     0.   ]
			 [ 0.    -0.2    0.     0.     0.    -0.818  0.     0.     1.     0.   ]
			 [ 0.     0.     0.     0.     0.     0.     0.091  1.    -1.     0.083]
			 [ 0.     0.     0.     0.     0.     0.     0.    -0.727  0.     0.   ]
			 [ 0.     0.     0.     0.     0.     0.     0.     0.    -0.727  0.   ]
			 [ 0.     0.     0.     0.     0.     0.     0.2    0.     0.    -0.818]

			 [ 0.          0.10890899 -0.83590899  0.10890899 -0.83590899 -0.83590899
  0.10890899 -0.727      -0.727      -0.727     ] 


			 smallest eigenvalue -0.8359089859433243 


			vp = 0.09090909090909091 


			here -0.164091014056